In [15]:
import pandas as pd
import pyodbc 
import numpy as np


In [16]:
#initial cleaning function
def clean_sheet(dataframe):
    adj=dataframe.dropna(axis=1,how='all')
    adj=dataframe.set_index(adj.columns[0]) #use stock tickers as the index
    return adj

In [24]:
def fundamental_df_clean(region,tabs,volume_low,size_low):
    data_dict={}

    for s in tabs:
        df=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result data\{0} 2007-2017.xlsx'.\
                                     format(region),sheet_name=s))
        data_dict[s]=df

    # select the right universe for US
    universe_evaluate=(data_dict["Market cap"]>size_low)&(data_dict["Volume"]>volume_low)
    data_dict["Revision"]=data_dict["Revision"][universe_evaluate]
    data_dict["Market cap"]=data_dict["Market cap"][data_dict["Market cap"]>size_low]    
    
    
    fundamental_df=data_dict["Date"].unstack(-1).to_frame()
    fundamental_df.index=fundamental_df.apply(lambda x: x.name[1],axis=1)
    fundamental_df.columns=["Date"]
    
    for i in tabs:
        try:
            fundamental_df[i]=data_dict[i].unstack(-1).values
        except ValueError:
            pass
        
    fundamental_df=fundamental_df.dropna(how="all",axis=0)
    fundamental_df=fundamental_df[-np.isnan(fundamental_df["Revision"])]
    fundamental_df=fundamental_df[-np.isnat(fundamental_df.Date)]
    fundamental_df["Supersector"]=fundamental_df.apply(lambda x:data_dict["Property"].loc[x.name,"Supersector"]\
                                               if type(x.name)==str else None,axis=1)
    return fundamental_df

In [25]:
US=fundamental_df_clean('US',['Date','Revision','Market cap','Volume','Revision vol','Property'],3,500)

In [27]:
Europe=fundamental_df_clean('Europe',['Date','Revision','Market cap','Volume','Revision vol','Property'],3,500)

In [29]:
Europe.to_csv('Europe.csv')